In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model

from utils.custom_loss import mae_wrap_angle, mse_wrap_angle, angle_diff_deg
from utils.load_data_raw import DataGenerator_raw, load_raw_all_h5
from utils.dataset_split import *
from utils.eval import *
from utils.utils import get_filelist, open_json

NUM_WORKER = 4
custom_obj={'mse_wrap_angle': mse_wrap_angle, 'mae_wrap_angle': mae_wrap_angle}

In [ ]:
# load filelist
model_dir = '../data/models_trained/'
filelist = get_filelist(model_dir)

# load data
dset_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10_scaledMM.h5'
feat, targ, ID_ref, pos_t, cond_t, par = load_raw_all_h5(dset_dir)

# load unscaled data
dset_nsc_dir = '/media/feliximmohr/Storage/master_thesis/generated/database/raw/raw_nf10_mid/database_raw_nf10.h5'
feat_nsc, targ_nsc, _, _, _, _ = load_raw_all_h5(dset_nsc_dir)

# create parameter dicts for the test batch generators
params = create_test_params(feat, targ, par, shuffle=False)
params_nsc = create_test_params(feat_nsc, targ_nsc, par, shuffle=False)

part = {}
for i in range(len(cond_t)):
    cond = cond_t.iloc[i]
    cond_ids_test, pos_ids_test, subject_ids_test = get_subset_ids([cond[0]], cond_t)
    part[cond[0]] = get_subset_sample_idx(ID_ref, cond_ids_test, pos_ids_test, subject_ids_test)

In [ ]:
# extract model and corresponding history and test partition filenames from filelist
m_flist = [x for x in filelist if ('history' not in x) and ('partition_test' not in x)]
h_flist = ['_' for x in np.zeros(len(m_flist))]
pt_flist = ['_' for x in np.zeros(len(m_flist))]
for file in filelist:
        if ('history' in file):
            name = file.replace('_history.json', '.h5')
            h_flist[m_flist.index(name)] = file
        if ('partition_test' in file):
            name = file.replace('_partition_test.json', '.h5')
            pt_flist[m_flist.index(name)] = file

# seperate model list based on test type (topology/wrapping/normalization)
m_flist_tt = [x for x in m_flist if 'toptest' in x]
m_flist_nt = [x for x in m_flist if 'normtest' in x]
m_flist_wt = [x for x in m_flist if 'wraptest' in x]

h_flist_tt = [h_flist[m_flist.index(i)] for i in m_flist_tt]
h_flist_nt = [h_flist[m_flist.index(i)] for i in m_flist_nt]
h_flist_wt = [h_flist[m_flist.index(i)] for i in m_flist_wt]

pt_flist_tt = [pt_flist[m_flist.index(i)] for i in m_flist_tt]
pt_flist_nt = [pt_flist[m_flist.index(i)] for i in m_flist_nt]
pt_flist_wt = [pt_flist[m_flist.index(i)] for i in m_flist_wt]

In [ ]:
def print_filelist(flist):
    """Print tuple of filelists incl index."""
    if not isinstance(flist,tuple):
        flist = (flist,)
        
    for i in range(len(flist[0])):
        s = ' -' if i<10 else '-'
        for j in range(len(flist)):
            print(i, s, flist[j][i])

# print model list
#print_filelist((m_flist,h_flist,pt_flist))

In [ ]:
def load_m_h_pt(model_dir, m_flist, h_flist, pt_flist):
    """Load models from model list and corresponding history and test partition."""
    m = []
    h = ['_' for x in np.zeros(len(m_flist))]
    p = ['_' for x in np.zeros(len(m_flist))]
    for i,j in enumerate(m_flist):
        m.append(load_model(model_dir + j, custom_objects=custom_obj))
        if h_flist[i] != '_':
            h[i] = open_json(model_dir, h_flist[i])
        if pt_flist[i] != '_':
            p[i] = open_json(model_dir, pt_flist[i])
    return m, h, p

# load models
#m_nt, h_nt, pt_nt = load_m_h_pt(model_dir, m_flist_nt, h_flist_nt, pt_flist_nt)
#m_wt, h_wt, pt_wt = load_m_h_pt(model_dir, m_flist_wt, h_flist_wt, pt_flist_wt)
m_tt, h_tt, pt_tt = load_m_h_pt(model_dir, m_flist_tt, h_flist_tt, pt_flist_tt)

## Prediction Evaluation

In [ ]:
print_filelist(m_flist_tt)

In [ ]:
#part_x_M027, pos_ids = get_pos_IDs(part['NFCHOA_M027'],ID_ref)
#part_x_R006, _ = get_pos_IDs(part['NFCHOA_R006'],ID_ref)

params_t = params.copy()
params_t['batch_size'] = 1000

In [ ]:
def model_pred_pos(model, part, params, ID_ref):
    """Returns predictions on model based on test partition per position as list."""
    part_x, pos_ids = get_pos_IDs(part,ID_ref)
    b_gen = []
    pred = []
    y = []
    for j in range(len(part_x)):
        b_gen.append(DataGenerator_raw(part_x[j], **params))
        pred.append(model.predict_generator(b_gen[j], verbose=1, use_multiprocessing=True, workers=64))
        y.append(get_y_gen(b_gen[j]))
    return pred, y

In [ ]:
def plot_pred(pred, y):
    """TODO."""
    %matplotlib
    x =  np.kron(np.arange(3600), np.ones(20))
        
    #plot
    f, axes = plt.subplots(3, 4, sharey=True, sharex=True)
    axes[0, 1].scatter(x,pred[2])
    axes[0, 2].scatter(x,pred[1])
    axes[0, 3].scatter(x,pred[0])
    axes[1, 0].scatter(x,pred[6])
    axes[1, 1].scatter(x,pred[5])
    axes[1, 2].scatter(x,pred[4])
    axes[1, 3].scatter(x,pred[3])
    axes[2, 1].scatter(x,pred[9])
    axes[2, 2].scatter(x,pred[8])
    axes[2, 3].scatter(x,pred[7])
    #((ax0_,ax02,ax01,ax00),(ax06,ax05,ax04,ax03),(ax1_,ax09,ax08,ax07))
    
def plot_pred_pos(pred,y):
    %matplotlib
    x =  np.kron(np.arange(360), np.ones(200))
    plt.scatter(x,pred)

In [ ]:
pred, y = model_pred_pos(m_tt[14] , part['NFCHOA_M027'], params_t, ID_ref) 

In [ ]:
plot_pred_pos(pred[9],y)

In [ ]:
x =  np.kron(np.arange(360), np.ones(200))

In [ ]:
min(pred[0])

In [ ]:
min(y[0])